Training an embedding with a simple dataset where the word relationships are very clear.

In [1]:
# Needed to import from the data directories
import sys
import tensorflow as tf
import json

from pathlib import Path

parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))

In [2]:
from modules.utils import import_dataset, merge_datasets, _pre_process_sentence, visualize_embeddings
from modules.model import Word2Vec

with open("../data/eval_data_poke_and_types.json", "r") as f:
    POKEMONS_BY_TYPE = json.load(f)

In [3]:
easy = import_dataset('../data/train_data_evolutions_and_types.json')
hard = import_dataset('../data/train_data_pokedex_entries.json')

#dataset = merge_datasets(easy, hard[:1])
dataset = easy

In [22]:
window_size = 10
embedding_dim = 50

epochs = 100
# Use Adam optimizer with learning rate decay
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=1000,
        decay_rate=0.95
    )
)

# better with neg examples
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

word2vec = Word2Vec(dataset, embedding_dim)
train_dataset = word2vec.prepare_dataset(window_size=window_size, num_negative_samples=0)
word2vec.compile(optimizer, loss, metrics=['accuracy'])
word2vec.fit(train_dataset, epochs=epochs, verbose=1)

word2vec.summary()

Sample word pairs: [('bulbasaur', 'ivysaur'), ('bulbasaur', 'venusaur'), ('bulbasaur', 'grass'), ('bulbasaur', 'poison'), ('ivysaur', 'bulbasaur')]
Epoch 1/100
5/5 [==============================] - 1s 52ms/step - loss: 0.7075 - accuracy: 0.4857
Epoch 2/100
5/5 [==============================] - 0s 26ms/step - loss: 0.6319 - accuracy: 0.6929
Epoch 3/100
5/5 [==============================] - 0s 16ms/step - loss: 0.5747 - accuracy: 0.8286
Epoch 4/100
5/5 [==============================] - 0s 16ms/step - loss: 0.5256 - accuracy: 0.9161
Epoch 5/100
5/5 [==============================] - 0s 22ms/step - loss: 0.4826 - accuracy: 0.9661
Epoch 6/100
5/5 [==============================] - 0s 18ms/step - loss: 0.4446 - accuracy: 0.9821
Epoch 7/100
5/5 [==============================] - 0s 10ms/step - loss: 0.4106 - accuracy: 0.9929
Epoch 8/100
5/5 [==============================] - 0s 10ms/step - loss: 0.3799 - accuracy: 0.9964
Epoch 9/100
5/5 [==============================] - 0s 10ms/step - lo

In [23]:
# get the embeddings matrix via the layer
word2vec.get_layer("word_embedding").get_weights()[0].shape


(128, 50)

In [24]:
# Load and organize evaluation data

all_pokemons = []
for pokemons in POKEMONS_BY_TYPE.values():
    all_pokemons.extend(pokemons)

pokemon_to_type = {}
for type_name, pokemon_list in POKEMONS_BY_TYPE.items():
    for pokemon in pokemon_list:
        pokemon_to_type[pokemon] = type_name

poke_types = [pokemon_to_type[poke] for poke in all_pokemons]
poke_and_types = {poke: poke_type for poke, poke_type in zip(all_pokemons, poke_types)}

# Remove those that aren't in the vocabulary
pokes_to_remove = []
for pokemon in poke_and_types.keys():
    if not (pokemon in word2vec.vocabulary):
        pokes_to_remove.append(pokemon)         

for poke in pokes_to_remove:
    poke_and_types.pop(poke)

In [25]:
visualize_embeddings(word2vec, dim=2, words=poke_and_types, title='Word2Vec Pokembeddings',) 

In [29]:
from modules.test_types import test_script
results = test_script(word2vec)

Pass: 98.0% (294 of 300)
 Fail: 2.0% (6 of 300)


In [30]:
results.fail_counter

6

In [19]:
# Save the model to a directory
model_save_path = "./saved_model"
word2vec.save(model_save_path)

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets
